# Mount drive and append path to PYTONPATH


In [ ]:
import os
import sys

from google.colab import drive

drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/DeepLCMS/train_google_colab")

# Import and install libraries

In [ ]:
%%capture
!pip install lightning
!pip install timm
!pip install torchinfo

In [ ]:
import gc
from typing import Optional, Tuple

import colab_functions
import colab_utils
import pandas as pd
import prepare_data
import pytorch_lightning as pl
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
import train_NN
from google.colab import drive
from lightning.pytorch.loggers import CSVLogger
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import Callback, EarlyStopping
from pytorch_lightning.trainer.trainer import Trainer
from timm import create_model
from torchmetrics.classification import (
    BinaryAUROC,
    BinaryF1Score,
    BinaryPrecision,
    BinaryRecall,
)

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Unzip data

In [ ]:
!unzip -q experiment.zip

# Check if GPU is used

In [ ]:
device = colab_functions.get_device()

# Evaluate the test set


In [ ]:
preprocess_test = timm.data.create_transform(**data_cfg, is_training=False)

test_data = datasets.ImageFolder(
    root=test_dir,
    transform=preprocess_test,
    target_transform=None,
)

test_dataloader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    drop_last=False,
    pin_memory=True,
)

model.eval()
predictions = trainer.predict(model, test_dataloader)

In [ ]:
all_labels = torch.tensor(test_dataloader.dataset.targets)
all_labels

In [ ]:
probabilities = torch.sigmoid((torch.cat(predictions, dim=0)))

# Threshold probabilities to get binary predictions (0 or 1)
threshold = 0.5
binary_predictions = (probabilities > threshold).float().view(-1)
binary_predictions

In [ ]:
acc = (all_labels == binary_predictions).sum().item() / len(all_labels)


metric_f1 = BinaryF1Score()
f1 = metric_f1(all_labels, binary_predictions)


bcm = BinaryConfusionMatrix()
bcm(all_labels, binary_predictions)
fig_, ax_ = bcm.plot()